In [33]:
import json
import pandas as pd
from elasticsearch import Elasticsearch
import mariadb
from pyhive import hive
from datetime import datetime

# **1-HIVE**

Vamos a sacar todas las victorias de Fernando Alonso, y deberia de devolvernos 32 resultados.

In [3]:
conn = hive.Connection(host="localhost", username="hive", database="f1_hive")
cursor = conn.cursor()

In [4]:
cursor.execute('''
SELECT rs.name, rs.year
FROM hive_results r
JOIN hive_drivers d ON r.driverid = d.driverid
JOIN hive_races rs ON r.raceid = rs.raceid
WHERE d.driverref = 'alonso'
  AND r.position = 1
               ''')

results = cursor.fetchall()
print(results)
print(len(results))

[('"Singapore Grand Prix"', 2008), ('"Japanese Grand Prix"', 2008), ('"Malaysian Grand Prix"', 2007), ('"Monaco Grand Prix"', 2007), ('"European Grand Prix"', 2007), ('"Italian Grand Prix"', 2007), ('"Bahrain Grand Prix"', 2006), ('"Australian Grand Prix"', 2006), ('"Spanish Grand Prix"', 2006), ('"Monaco Grand Prix"', 2006), ('"British Grand Prix"', 2006), ('"Canadian Grand Prix"', 2006), ('"Japanese Grand Prix"', 2006), ('"Malaysian Grand Prix"', 2005), ('"Bahrain Grand Prix"', 2005), ('"San Marino Grand Prix"', 2005), ('"European Grand Prix"', 2005), ('"French Grand Prix"', 2005), ('"German Grand Prix"', 2005), ('"Chinese Grand Prix"', 2005), ('"Hungarian Grand Prix"', 2003), ('"Bahrain Grand Prix"', 2010), ('"German Grand Prix"', 2010), ('"Italian Grand Prix"', 2010), ('"Singapore Grand Prix"', 2010), ('"Korean Grand Prix"', 2010), ('"British Grand Prix"', 2011), ('"Malaysian Grand Prix"', 2012), ('"European Grand Prix"', 2012), ('"German Grand Prix"', 2012), ('"Chinese Grand Prix"

In [5]:
cursor.close()
conn.close()

# **2-ELASTICSEARCH**

In [ ]:
es = Elasticsearch("http://localhost:9200")

res = es.search(index='my_index', body={
     'query': {
         'range': {
             'position': {
                 'gte': 1,
                 'lte': 3
             }
         }
     },
     'size': 1
})

# **3-GRAPHDB**

# **4-QUERIES COMPUESTAS**

In [56]:
conn = hive.Connection(host="localhost", username="hive", database="f1_hive")
cursor = conn.cursor()

In [58]:
cursor.execute('''
SELECT d.forename, d.surname
FROM hive_results r
JOIN hive_drivers d ON r.driverid = d.driverid
JOIN hive_races rs ON r.raceid = rs.raceid
WHERE rs.name = '"Spanish Grand Prix"'
  AND rs.year = 2024
  AND r.position = 1
               ''')

results = cursor.fetchall()
print(results)
fullname = results[0][0] + " " + results[0][1]
print(fullname)

[('Max', 'Verstappen')]
Max Verstappen


In [59]:
cursor.close()
conn.close()

In [ ]:
# Parámetros
index = "my_index"

# Consulta usando match_phrase
query = {
    "query": {
        "term": {
            "pilot_name": fullname
        }
    }, 
    "aggs": {
        "avg_position": {
            "avg": {
                "field": "position"
            }
        }
    },
    "size": 100
}

# Ejecuta la consulta
response = es.search(index=index, body=query)

print(json.dumps(response.body, indent=4))

{
    "took": 60,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 87,
            "relation": "eq"
        },
        "max_score": 2.3753045,
        "hits": [
            {
                "_index": "my_index",
                "_type": "_doc",
                "_id": "36",
                "_score": 2.3753045,
                "_source": {
                    "ranking_date": "2024-09-04",
                    "pilot_name": "Max Verstappen",
                    "position": 6,
                    "pilot_review": "Verstappen and Red Bull were on the back foot throughout the Monza weekend, with the three-time world champion absent from both the battle for pole position and the race victory amid the RB20\u2019s ongoing issues. After coming home sixth in a commendable damage limitation exercise, Verstappen bluntly commented that the team need to \u201cbasi